In [1]:
import chromadb
import langchain
import ollama
import os
import streamlit
import textract # appears broken for pdf on windows due to shell call
import pypdf



In [2]:
# set variables to point at documents
document_1_location = "C:/Users/justi/Documents/GitHub/SmallProjects/documents/Scotland's Wild Deer_ A National Approach 2015-2020 Priorities.pdf"
document_2_location = "C:/Users/justi/Documents/GitHub/SmallProjects/documents/deer-management-on-scotlands-national-forest-estate.pdf"

In [3]:
def extract_text(document_location: str) -> str:
    extension = os.path.splitext(document_location)[-1].lower()
    if extension == '.txt':
        with open(document_location) as document:
            full_text = document.read()
    elif extension == '.pdf':
        # https://pypdf.readthedocs.io/en/stable/user/extract-text.html
        reader = pypdf.PdfReader(document_location)
        number_of_pages = len(reader.pages)
        full_text = ""
        # extract text page by page
        for page in reader.pages:
            text = page.extract_text()
            full_text = full_text + text
        # remove linebreaks
        full_text = full_text.replace('\n', ' ')
    else:
        # try to use textract - warning may be buggy - certainly fails pdfs
        # https://textract.readthedocs.io/en/stable/
        full_text = textract.process(document_location)
    return full_text 

In [4]:
# # set up our ChromaDB NON-PERSISTENT database
# # https://docs.trychroma.com/getting-started
# chroma_client = chromadb.Client()
# # while we do not intend to persist this chrmoadb, lets use a careful creation function
# collection = chroma_client.get_or_create_collection(name="comparison_db")

In [5]:
# extract our documents to plain text
text_1 = extract_text(document_1_location)
text_2 = extract_text(document_2_location)

In [6]:
# # chunk our text to keep it inside the context window
# # https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter/
# text_splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(
#     chunk_size=2000, # characters not words
#     chunk_overlap=100,
#     length_function=len,
# #     is_separator_regex=False,
# )
# text_1_chunks = text_splitter.split_text(text_1)
# text_2_chunks = text_splitter.split_text(text_2)


In [7]:
# # store the chunks of one document in the database
# for index, chunk in enumerate(text_1_chunks):
#     collection.add(
#         documents=[chunk],
#         ids=[str(index)]
#     )
#     print(index, chunk)
    
# # stores a tarball at C:\Users\justi\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz

In [8]:
# # now lets perform a comparison between chunks document 2 and the most similar chunks of document 1 as found in the database
# answers = []
# for chunk in text_2_chunks:
#     print("CHUNK+++++++++++++++++++++++++++++++++++++++++++++++++")
#     print (chunk)
#     # retrieve the most similar chunk from the database
#     results = collection.query(
#         query_texts=[chunk],
#         n_results=1
#     )
#     print(results)
#     # join the top 2 results?
#     result = results['documents'][0][0] #+ " " + results[1]
#     print("MATCHED-------------------------------------------")
#     print(result)
#     query = f"""
#         Please describe 3 similarities and 3 differences between the following two passages. 
#         Provide these answers as two numbered lists labelled SIMILARITIES and DIFFERENCES.
        
#         Passage 1:
#         {chunk}
        
#         passage 2:
#         {result}
#         """
#     response = ollama.chat(model='llama3:8b', messages=[
#       {
#         'role': 'user',
#         'content': query,
#       },
#     ])
#     answer = response['message']['content']
#     print("ANSWER-------------------------------------------")
#     print(answer)
#     answers.append(answer)
    
    

In [9]:
# answers

In [10]:
# def recursive_summariser(
#     text: str,
#     model: str ='llama3:8b',
#     input_chunk_tokens: int = 5000, # characters not tokens
#     chunk_overlap: int = 20,
#     compression_ratio: float = 5.0,
#     max_summary_length: int = 5000, # characters not tokens
# ):
#     input_length = len(text)
#     print("Input_text_length: " + str(len(text)))
#     # see if we can achieve our desired compressions ratio with a less agressive compression ratio
#     required_compression = input_length / max_summary_length
#     if required_compression < compression_ratio:
#         # this could be a higher compression ration than desired, but unlikely
#         # it should ensure sufficient compression on late rounds to make the last round of compression close to optimal
#         compression_ratio = required_compression * 1.3 
#         print(f"changed compressions ratio to : {compression_ratio}")
#     # split the passage based on the token length
#     text_splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(
#         chunk_size=input_chunk_tokens, # characters not words
#         chunk_overlap=chunk_overlap,
#         # add full stops and commas to the separators to try to get the most sensible splitting 
#         separators=["\n\n", "\n", ".", ",", " ", ""],
#         length_function=len,
#     #     is_separator_regex=False,
#     )
# #     text_token_count = text_splitter.count_tokens(text)
# #     print("Starting tokens for this recursion:" + str(text_token_count))
#     text_chunks = text_splitter.split_text(text)
#     # handle the edge case that we are already down to a chunk smaller than the chunk size
#     if len(text_chunks) <= 1:
#         return text
#     # summarise each chunk in turn
#     responses = []
#     for chunk in text_chunks:
#         word_count = len(chunk.split(" "))
#         desired_word_count = int(word_count//compression_ratio)
#         print(f"chunk length: {len(chunk)}, word count: {word_count}, desired word count: {desired_word_count}")
#         query = f"""
# You are a professional document summariser
# Please summarise the following text from {word_count} words, down to {desired_word_count} words. 
# Be careful to retain as much of the overall meaning of the text as possible in your summary. 
# Include nothing but the summary in your reply. Do not say how many words it is summarised to or mention that it is a summary.

# Text:
# {chunk}
# """
#         response = ollama.chat(model='llama3:8b', messages=[
#           {
#             'role': 'user',
#             'content': query,
#           },
#         ])
#         response_text = response['message']['content']
#         responses.append(response_text)
    
#     # concatenate the chunks
#     summary = "\n".join(responses)
#     print("Summary length in characters: " + str(len(summary)))
    
#     # check summary length
#     summary_splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(
#         chunk_size=max_summary_length, # characters not words
#         chunk_overlap=0,
#         length_function=len,
#     #     is_separator_regex=False,
#     )
# #     summary_token_count = summary_splitter.count_tokens(summary)
#     summary_chunks = text_splitter.split_text(summary)
#     # if summary is short enough return it
#     if len(summary_chunks) <= 1:
#         return summary
#     # otherwise call recursive_summariser on the summary
#     else:
#         return recursive_summariser(summary)
    
# def tidy_text(text: str):
#     query = f"""
# You are a professional copy editor
# Please edit the following text ensuring that it maintains a consistent grammatical style throughout.
# Remove any references to summarisation or word counts.
# Ensure you preserve all the factual meaning, but remove any incoherent text.
# Break the text into meaningful paragraphs.
# Your edited text should be approximately the same number of words as the original.
# Your edited text must be written in British rather than American English.
# Include nothing but the edited text in your reply. Do not mention that it is edited.

# Text:
# {text}
#             """
#     edited_summary = ollama.chat(model='llama3:8b', messages=[
#       {
#         'role': 'user',
#         'content': query,
#       },
#     ])
#     return edited_summary['message']['content']

In [11]:
# raw_summary_1 = recursive_summariser(text_1)

In [12]:
# print(raw_summary_1)

In [13]:
# print(len(raw_summary_1))

In [14]:
# tidy_summary_1 = tidy_text(raw_summary_1)

In [15]:
# print(tidy_summary_1)

In [16]:
# print(len(tidy_summary_1))

In [17]:
# print(len(text_1))

In [18]:
# raw_summary_2 = recursive_summariser(text_2)

In [19]:
# print(raw_summary_2)

In [20]:
# print(len(raw_summary_2))

In [21]:
# tidy_summary_2 = tidy_text(raw_summary_2)

In [22]:
def compare_summaries(
    summary_1: str,
    summary_2: str,
    model:str = 'llama3:8b',
    max_length:int = 10000,
):
    query = f"""
Please describe the similarities and differences between the following two passages. 
Focus on similarites and differences of content and emphasis, rather than style or phrasing.
Keep your answer to a maximum of 1000 words in total.
Provide your answers as two numbered lists labelled SIMILARITIES and DIFFERENCES.

Passage 1:
{summary_1}

passage 2:
{summary_2}
"""
    if len(query) > max_length:
        print("Warning query may exceed context window")
    response = ollama.chat(model=model, messages=[
      {
        'role': 'user',
        'content': query,
      },
    ])
    answer = response['message']['content']
    return(answer)

In [25]:
comparison = compare_summaries(text_1, text_2, 'herald/phi3-128k:latest')
print(comparison)

Warning query may exceed context window
Scotland's Deer Management Strategy outlines the comprehensive approach towards managing deer populations within Scotland. The strategy emphasizes sustainable practices, conservation of biodiversity, and enhancement of landscapes. 

The strategy includes various management practices such as monitoring wildlife populations and their habitats, implementing measures to prevent overgrazing and protect ground vegetation. It also focuses on promoting public engagement through education programs about deer ecology and the importance of sustainable forestry practices. Moreover, it highlights the benefits of healthy deer populations for maintaining a balanced ecosystem.


In [26]:
def compare_strategies(
    summary_1: str,
    summary_2: str,
    model:str = 'llama3:8b',
    max_length:int = 10000,
):
    query = f"""
You are an enviromental strategy expert.
Please describe the similarities and differences in the strategies outlined by the following passages. 
Focus on similarites and differences of strategy, rather than style, tone or phrasing.
Provide your answers as two numbered lists labelled SIMILARITIES and DIFFERENCES.

Passage 1:
{summary_1}

passage 2:
{summary_2}
"""
    if len(query) > max_length:
        print("Warning query may exceed context window")
    response = ollama.chat(model=model, messages=[
      {
        'role': 'user',
        'content': query,
      },
    ])
    answer = response['message']['content']
    return(answer)

In [29]:
strategy_comparison = compare_strategies(text_1, "help", 'herald/phi3-128k:latest')
print(strategy_comparison)

Warning query may exceed context window


ResponseError: model 'herald/phi3-128k:latest' not found, try pulling it first